In [ ]:
import numpy as np
import pyroomacoustics as pra
import matplotlib.pyplot as plt
import librosa
import soundfile as sf
import webrtcvad
import sounddevice as sd
import pandas as pd
import os

In [ ]:
def npow2 (x) :
    # finds the next power of 2 of the input x
    return 1 <<(x -1) . bit_length ()

def stftprep (x , fs , time ):
    # splits the input array in an array of frames of fixed length (to be used to prepare
    #array for VAD or STFT )
    
    #x = input array
    #fs = sampling frequency
    # time = time of the split frames ( determines the frame length )

    # returns the array of the split input data
    frames = int( time * fs )
    X = np.array([ x[i: i+ frames] for i in range(0, len(x)-frames, frames)])
    return X

def VAD(aggr , data ,sr , time ):
    #This function uses the webrtcvad module to do voice activity of detection
    #on a numpy array of audio data and returns the array of the voiced segments
    
    
    # aggr how sensitive voice detection is with options 0(least),1,2,3(most)
    # data ( numpy array ) to be classified as voiced / unvoiced
    #sr is the sampling rate
    # the time of the frame in ms (10 ,20 ,30 ms)

    # Sets the vad object
    vad = webrtcvad . Vad ( aggr )
    # sets frame length
    framesz = time /1000
    # cuts the data in the frame size
    Z = stftprep ( data , sr , time /1000)
    # returns indexes of voiced data
    ind = [i for i in range ( len (Z) ) if vad . is_speech ( np . int16 ( Z[i ]*32768) . tobytes () ,sr )]
    # multiply by 32768
    # creates the array of the voiced parts of the data
    emin = [Z [k] for k in ind ]
    eminor = np . array ([ j for i in emin for j in i ])
    return eminor

def gcc_phat ( sig , refsig , fs , max_tau = None , interp =4) :
    '''
    I modified this function from the original :
    Copyright (c) 2017 Yihui Xiong
    Licensed under the Apache License , Version 2.0 ( the " License ");
    you may not use this file except in compliance with the License .
    You may obtain a copy of the License at
    http :// www . apache . org/ licenses / LICENSE -2.0
    '''
    
    #This function computes the offset between the signal sig and the reference signal refsig
    #using the Generalized Cross Correlation - Phase Transform (GCC - PHAT ) method.

    # make sure the length for the FFT is larger or equal than len ( sig ) + len ( refsig )
    n = sig.shape[0] + refsig.shape[0]

    # find next power of 2 to make FFT faster
    n = npow2(n)

    # Generalized Cross Correlation Phase Transform
    # Uncomment to add window , multiply the signals sig and refsig with the window
    # window = np. hamming ( len ( sig ))

    SIG = np.fft.rfft(sig, n=n)
    REFSIG = np.fft.rfft(refsig , n=n)

    R = SIG * np.conj(REFSIG)
    # phat weighting
    ab = np.abs(R)
    ab [ab<1e-10] = 1e-10
    # obtains gcc
    cc = np.fft.irfft(R/np.abs(ab),n=npow2(interp*n))

    # Uncomment below to normalize
    #cc = cc/np. max (cc)

    max_shift = int(interp*n/2)
    if max_tau:
        max_shift = np.minimum(int(interp*fs*max_tau),max_shift)
    
    cc = np.concatenate((cc[-max_shift:], cc[:max_shift +1]) )
    # find max cross correlation index ( TDOA )
    shift = np.argmax(np.abs(cc)) - max_shift

    tau = shift/float(interp*fs )

    return tau , cc

def myinwhole ( Dat ,fs , mics = 6) :
    # Function that generates the GCC - PHAT function for all different microphone pair combinations

    # Dat = input audio data ( data must be of shape mics X any audio length )
    #fs is the sampling rate
    #mics is the number of microphone (defaults in 6 for the respeaker)
    ind = list(itertools.combinations(range(mics),2))
    GCC = []

    for i,j in ind:
        t,c = gcc_phat(Dat[i], Dat[j], fs, 0.0003)

    GCC.append(c)

    GCC = np.array(GCC)
    GCC = np.transpose(GCC)
    # Change the 4 below with the interpolation factor given in gccphat () function
    G = -len(c)/(2*4*fs)
    #X and Y are included to be able to print the input image with matplotlib.pyplot.colormesh (X,Y, GCC )
    Y = np.linspace(G,-G,len(c)+1)
    X = np.linspace(1,len(ind),len(ind)+1)
    return X , Y , GCC

def myin(Dat,frame) :
    # Function that generates the GCC - PHAT function for all different microphone pair combinations
    
    # Dat = input audio data ( data must be of shape mics X any audio length )
    #fs is the sampling rate
    #mics is the number of microphone (defaults in 6 for the respeaker)
    ind = [[0,1],[0,2],[0,3],[0,4],[0,5],[1,2],[1,3],[1,4],[1,5],[2,3],[2,4],[2,5],[3,4],[3,5],[4,5]]
    GCC = []

    for i,j in ind:
        t,c = gcc_phat(Dat[i][frame], Dat[j][frame], fs, 0.0003)
        GCC.append(c)

    GCC = np.array(GCC)
    GCC = np.transpose(GCC)
    # Change the 4 below with the interpolation factor given in gccphat () function
    G = -len(c)/(2*4*fs)
    #X and Y are included to be able to print the input image with matplotlib.pyplot.colormesh (X,Y, GCC )
    Y = np.linspace(G,-G,len(c)+1)
    X = np.linspace(1,len(ind),len(ind)+1)
    return X , Y , GCC

def roomsimdata(rt60,dim ,fs,data,sloc,miccent,mich,deg,delay =1,snr = None , N =1):
    
    #Function that simulates a room and returns the signals captured by the microphones
    
    #rt60 is the time it takes for reverberations to fall below 60dB
    #dim is a 3D array having the xyz dimensions of the room
    #fs is the sampling frequency
    #data is the length N array of the wav data of the source
    #sloc is a Nx3 array of the source location in xyz
    #delay is how much to wait before start playing the source
    #miccent is a 2D array of the center of the mic array
    #mich is the height at which the mic array is placed
    #deg is an array with the degrees of each source
    #snr is the SNR at the mic signals (default is set to None)
    #N is the number of sources
    
    
    e_absorption, max_order = pra.inverse_sabine(rt60, dim)
    # creates room object
    room = pra.ShoeBox(
        dim, fs, materials=pra.Material(e_absorption), max_order=max_order
    )

    
    #adds sources to the room
    for i in range(N):
        room.add_source(sloc[:,deg[i]], signal= data[i], delay = delay)
    
    #generates mic lcoations
    G = pra.beamforming.circular_2D_array( miccent , 6 , 0 , 0.0463 )
    mic = []


    for i in range(0,6):
        c = [ G[0][i],G[1][i], mich]
        mic.append(c) 

    mic_locs = np.c_[mic[0],mic[1],mic[2],mic[3],mic[4],mic[5]]   
    #adds the microphone locations in the room
    room.add_microphone_array(mic_locs)
    
    # finds the room impulse responses
    room.compute_rir()
    #r = room.rir[1][0]
    #plt.plot(np.arange(len(r)) / room.fs, r)
    #plt.title("The RIR from source to mic 1")
    #plt.xlabel("Time /s")
    
    #simulates the room for a given SNR
    room.simulate(snr)
    
    #returns the signals captured by the microphone array
    return room.mic_array.signals

def myvad(data ,sr ,time, th):
    #This function performs voice activity of detection with an energy threshold
    
    # data (numpy array) to be classified as voiced / unvoiced
    #sr is the sampling rate
    # time is the time of the frames
    #th is the energy threshold above which a frame is considered voiced
    
    Z = [stftprep(data[i], sr, time/1000) for i in range(6)]
    ind = [i for i in range(len(Z[0])) if np.sum(np.square(Z[0][i]))/len(Z[0]) > th and np.sum(np.square(Z[1][i]))/len(Z[0]) > th and np.sum(np.square(Z[2][i]))/len(Z[0]) > th and np.sum(np.square(Z[3][i]))/len(Z[0]) > th and np.sum(np.square(Z[4][i]))/len(Z[0]) > th and np.sum(np.square(Z[5][i]))/len(Z[0]) > th]
    Roo2 = []
    for l in range(len(Z)):
        emin = [Z[l][k] for k in ind]
        eminor = np.array([j for i in emin for j in i])
        Roo2.append(eminor)
    return Roo2

In [ ]:
#load speech data from AMI corpus
l ,fs = librosa.load("ed1a.wav", sr = 16000)
z ,fs = librosa.load("ed1c.wav", sr = 16000)
p ,fs = librosa.load("ed1d.wav", sr = 16000)

#do VAD and seperate them in 2 second utterances
time = 2
L = VAD(3,l,fs, 30)
L = stftprep(L,fs, time)

Z = VAD(3,z,fs, 30)
Z = stftprep(Z,fs, time)

P = VAD(3,p,fs, 30)
P = stftprep(P,fs, time)

G = np.concatenate((L,P,Z))
np.shape(G)

In [ ]:
mode = 0o777
os.mkdir('GCC',mode)

In [ ]:
#This part simulates a room for all of the specified degrees and obtains the GCCs 
#for different utterances to be used in learning

time = 150 #frame length in ms
threshold = 5.0149176589231265e-05 


#center of source (same as for microphone for this simulation)
source_center = [2,2]
#distance of source from microphone array
radius = 1.5

source_height = 1.5
source = pra.beamforming.circular_2D_array(source_center,360,0,radius)
#source = np.hstack((source[:,330:360],source[:,0:30])) #for 60 degrees
source = np.vstack([source, np.ones(360)*source_height])
rt60 = 0.4 #reverberation time
r_dimensions = [5,5,3] #room dimensions xyz
mic_center = [2,2] #microphone center
degrees = np.arange(0,360, 10) # degrees
mic_height = 1.5 # micorphone height
delay = 0.1 # delay after which source plays
snr = [None] #snrs
Nosources = 1 # have to define the numebr of sources
fs = 16000 #sampling rate
path = r"" #path where the training data is to be saved

for l in range(len(degrees)):
    df = pd.DataFrame()
    for j in range(len(snr)):
        for k in range(150):
            Room = roomsimdata(rt60,r_dimensions ,fs,[G[k]],source,mic_center,mic_height,[degrees[l]],delay,snr[j],Nosources) #change L,Z,P for different voice sets
            #Room = [Room[i] for i in range(6)]
            Voiced = myvad(Room,fs, time ,threshold)
            Voiced = [stftprep(Voiced[i],fs,time/1000) for i in range(6)]
            for m in range(len(Voiced)):
                X,Y,GCC = myin(Voiced,m)
                #plt.figure()
                #plt.pcolormesh(X,Y, np.abs(GCC), vmin=0, vmax= np.max(abs(GCC)))
                #plt.show()
                print(f'degree{l},snr {j}, word {k}')
                df = df.append(pd.DataFrame(GCC))
    df.to_csv(path+'GCC/gcc' + str(degrees[l])+'.csv')

In [ ]:
#This code simulates a room, plots the GCC of an utterance and plays the simulated sound received by a microphone
#to check if the simulation is representative of the real data

#center of source (same as for microphone for this simulation)
cent = [2,2]
#distance of source from microphone array
rad = [1.5,3,5]

#center of source (same as for microphone for this simulation)
source_center = [2,2]
#distance of source from microphone array
radius = 1.5

source_height = 1.5
source = pra.beamforming.circular_2D_array(source_center,360,0,radius)
#source = np.hstack((source[:,330:360],source[:,0:30])) #for 60 degrees
source = np.vstack([source, np.ones(360)*source_heigth])
rt60 = 0.4 #reverberation time
r_dimensions = [5,5,3] #room dimensions xyz
mic_center = [2,2] #microphone center
degrees = np.arange(0,360, 10) # degrees
mic_height = 1.5 # micorphone height
delay = 0.1 # delay after which source plays
snr = [None] #snrs
Nosources = 1 # have to define the numebr of sources
fs = 16000 #sampling rate
Roo = roomsimdata(rt60,dim ,fs,[G[1007]],source,miccent,mich,[degrees[0]],delay,snr[0],Nosources)

Dat = [Roo[i] for i in range(6)]
X,Y,GCC = myinwhole(Dat,fs)
plt.figure()
plt.pcolormesh(X,Y, np.abs(GCC), vmin=0, vmax= np.max(abs(GCC)))
plt.title('GCC-PHAT Input Matrix')
plt.xlabel('Mircophone Pairs')
plt.ylabel('Time Delay /s')
plt.show()

sd.play(Roo[0], fs)
status = sd.wait()